### Bert loading

In [15]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from sklearn.metrics import classification_report


In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

model_name = "bert-base-uncased" 

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Since we want to see how each word impact the final resutls , it makes mroe sense to have a classification taks 
Nope, BERT is not initially trained for classification.it is initially trained for MLM and NSP
For that we will fine tune bert on a sentiment analysis taks

## Fine tuning bert

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_train=pd.read_csv("/kaggle/input/sst2-dataset/Finalv SST-2 dataset CSV format/train.csv")
data_test=pd.read_csv("/kaggle/input/sst2-dataset/Finalv SST-2 dataset CSV format/test.csv")
data_val=pd.read_csv("/kaggle/input/sst2-dataset/Finalv SST-2 dataset CSV format/val.csv")

In [6]:
class SST2Dataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df['sentence'].tolist()   
        self.labels = df['label'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors=None 
        )
        
        item = {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }
        return item

Bert doesnt see words ,it sees ten tensors , and the dataset takes each sample and converts it to an item with three properties (inputs_ids,attention_mask ,labels) when we use input id is that we converted back each token to its token id that bert can understand

In [7]:
print(data_train.isnull().sum())
print(len(data_train))


label       0
sentence    0
dtype: int64
6920


In [8]:
data_train.iloc[5] 
# 1==positive
#0== bad bad 

label                                                       1
sentence    Campanella gets the tone just right -- funny i...
Name: 5, dtype: object

In [9]:
train_dataset = SST2Dataset(data_train, tokenizer)
val_dataset = SST2Dataset(data_val, tokenizer)
test_dataset = SST2Dataset(data_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [10]:
model = model.to(device)


In [11]:
optimizer = Adam(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

In [12]:
loss=CrossEntropyLoss()

In [13]:
epochs=5

In [16]:
model.train()
for epoch in range(epochs):
    print(f" the epoch is{epoch+1}")
    train_loss = 0
    correct_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct_train += (preds == labels).sum().item()
    
    train_acc = correct_train / len(train_dataset)
    print(f"Epoch {epoch+1} Train Loss: {train_loss/len(train_loader):.4f}, Acc: {train_acc:.4f}")
    model.eval()
    val_loss = 0
    correct_val = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            correct_val += (preds == labels).sum().item()
    
    val_acc = correct_val / len(val_dataset)
    print(f"Epoch {epoch+1} Val Loss: {val_loss/len(val_loader):.4f}, Acc: {val_acc:.4f}")

 the epoch is1
Epoch 1 Train Loss: 0.2814, Acc: 0.8842
Epoch 1 Val Loss: 0.2430, Acc: 0.9037
 the epoch is2
Epoch 2 Train Loss: 0.1089, Acc: 0.9620
Epoch 2 Val Loss: 0.2897, Acc: 0.8979
 the epoch is3
Epoch 3 Train Loss: 0.0304, Acc: 0.9918
Epoch 3 Val Loss: 0.3514, Acc: 0.9060
 the epoch is4
Epoch 4 Train Loss: 0.0118, Acc: 0.9971
Epoch 4 Val Loss: 0.3772, Acc: 0.9094
 the epoch is5
Epoch 5 Train Loss: 0.0202, Acc: 0.9931
Epoch 5 Val Loss: 0.3321, Acc: 0.9094


In [19]:
model.eval()

test_loss = 0
correct_test = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        test_loss += loss.item()
        preds = torch.argmax(logits, dim=1)

        correct_test += (preds == labels).sum().item()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_acc = correct_test / len(test_dataset)
avg_test_loss = test_loss / len(test_loader)

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
print(classification_report(all_labels, all_preds))

Test Loss: 0.2938, Test Accuracy: 0.9171
              precision    recall  f1-score   support

           0       0.92      0.91      0.92       912
           1       0.91      0.92      0.92       909

    accuracy                           0.92      1821
   macro avg       0.92      0.92      0.92      1821
weighted avg       0.92      0.92      0.92      1821



In [20]:
torch.save(model.state_dict(), "bert_sst2.pth")
